# JupyMail

Written by: [Mohajerani, Kamyar](mailto:mmohajer@gmu.edu)
    
Please feel free to check out the latest version and share bugs, issues, feature requests, or PRs on the [github page](https://github.com/kammoh/JupyMail).

This Jupyter notebook was written to allow one to import data from an excel spreadsheet, similar to MailMerge in Microsoft Office.  Given that it is written in python, there is much more flexibility in what you would like to do, if you are accomplished in Python.

In this basic version an excel spreadsheet is imported that contains all information needed in the email (the email address is the only mandatory field).  In addition, an email template is read into the notebook along with the name of the file to be used as an attachment (if desired).

To customize this mailer for your use, all that you need to do is make the appropriate changes in the _Settings Cell_.  More specifically, you will need to:

1. Specify the name of the excel spreadsheet file in `excel_file` and define what the fields are that will be used in your template in `excel_columns`. In this version, there are four columns, `First Name`, `Last Name`, `Email`, and `Degree`.  The  `Email` field is __mandatory__, and it can appear in any column. You can change the contents, the order, and number of columns in `excel_columns`.
2. Provide the name of the email template that will be mailed in `message_template_file'.
3. Give the name of the file to be used as an attachment (if any) in `attachment_file'.  If no attachment, put _None_.
3. Provide your outlook `username`, such as dr_python@gmu.edu, along with `your_name` such as _Monson Hayes_ that will be used in the FROM line of the email, and the email SUBJECT line in `email_subject`.



In [1]:
import sys
print(sys.version)

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


## Dependencies

In [2]:
#install openpyxl package
%pip install -U openpyxl

#install html2text package
%pip install -U html2text

# Note: if this didn't work, please try with adding '--user' switch 
#    or, try installing the dependency packages from command line
#    using pip or your Python package manager (e.g. conda)


# Note: You probably need to restart the kernel at this point!

Requirement already up-to-date: openpyxl in /anaconda3/lib/python3.7/site-packages (3.0.3)
Note: you may need to restart the kernel to use updated packages.
Requirement already up-to-date: html2text in /anaconda3/lib/python3.7/site-packages (2020.1.16)
Note: you may need to restart the kernel to use updated packages.


## Settings

In [56]:

username = 'mmohajer@masonlive.gmu.edu' # Email account e.g. hayes@gmu.edu

your_email = username
your_name = "Monson Hayes" # change this to your name


# All files need to be present in the script directory, otherwise specify the full path
excel_file ='ExcelEmailList.xlsx'
# Available columns in the Excel file can be substituted into the message_template.
# 'EMAIL' field is mandatory
excel_columns = ['FIRST_NAME', 'LAST_NAME', 'EMAIL', 'DEGREE']

message_template_file = 'message_template.html' # Change this to the html file for your email message template
                                                # A file with '.html' extension creates an HTML message
                                                #   accompanied with a simplified Markdown version of the same text
                                                #   for email clients that do not support HTML.
                                                # A file with '.txt' extension creates a plain-text message

email_subject = "Emailer"  # Put what you want to be in the email subject line here.

attachment_file = 'attachment.pdf'  # Here you put in the name of an attachment, or put None.       

## Recipients

In [49]:
from openpyxl import load_workbook
from openpyxl.cell.read_only import EmptyCell
from IPython.display import HTML, display

def read_contacts(filename, columns, sheet=None):
    wb = load_workbook(filename=filename, read_only=True)

    # grab the requested sheet or active worksheet if none given
    ws = wn[sheet] if sheet else wb.active
    
    return [dict(zip(columns, [cell.value for cell in row[0: len(columns)] if cell.value])) for row in ws.rows if not any(isinstance(cell, EmptyCell) for cell in row)]


contacts = read_contacts(excel_file, excel_columns)

display(HTML(
   '<table><tr><th>{}</th></tr><tr>{}</tr></table>'.format(
       '</th><th>'.join(excel_columns),
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in contact.values())) for contact in contacts)
       )
))

FIRST_NAME,LAST_NAME,EMAIL,DEGREE
kamyar,mohajerani,kammoh@gmail.com,Computer Engineering
Kamy,m,kamyar@ieee.org,Computer Engineering
Kamyar,Mohajerani,mmohajer@gmu.edu,Computer Engineering


## Template

In [57]:
from string import Template
import getpass
from IPython.display import Markdown as md

def read_template(filename):
    with open(filename, 'r', encoding='utf-8') as template_file:
        template_file_content = template_file.read()
    return Template(template_file_content)


message_template = read_template(message_template_file)

def format_contact(contact):
    contact['FIRST_NAME'] = contact['FIRST_NAME'].title()
    contact['LAST_NAME'] = contact['LAST_NAME'].title()
    return contact


# This should print a preview of the template, substituting fields using the first row of the Excel file
md(message_template.substitute(format_contact(contacts[0])))

<body style="margin: 0; padding: 0;">

<p> Dear Kamyar, </p>

<p> I am very excited to hear that you have been admitted to George Mason University and are interested in pursuing a BS degree in Computer Engineering.  Our department has many outstanding faculty who are working on some amazing research projects and, at the same time, are winning the highest awards that the University and professional organizations have to offer for excellence in teaching.</p>

<p>Whether you want to get involved in projects that involve swarming blimps, robotic fish, machine learning, hardware security, 5G networks, satellite communications, the power grid, or robotics, Electrical and Computer Engineering is the place to be at Mason. I have attached a copy of the Fall 2019 ECE Newsletter that you may enjoy reading.</p>
	
<p>We all know and appreciate what a difficult and important decision it is to select the right University for you to earn your Bachelor's degree.  Wherever you decide to go, it will be the right decision if you take advantage of everything the University has to offer.  And we have a lot to offer at Mason.</p>
	
<p>We are here to help you with your decision, and if you matriculate to Mason, we have outstanding advisors who will help advise you and guide you through the program. If you have any questions, please free to send an email to 
	<a href="mailto:ece@gmu.edu"> ece@gmu.edu</a>.  
	If you would like to talk to someone in person, let us know and we will give you a phone call.  
	Please fee free to contact me directly if you wish.</p>
 
<p>We look forward to welcoming you to ECE at Mason in the Fall.</p>
	

Sincerely,<br><br>
<em>Monson H. Hayes<br>
	Professor and Chair<br>
	Department of Electrical and Computer Engineering</em>
</body>


## Messages

In [58]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email import encoders
from email.mime.base import MIMEBase
import html2text

def attachment_part(filename):
    with open(filename, "rb") as attachment:
        # must be application/octet-stream
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())   
    encoders.encode_base64(part)
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )
    return part

def create_messages(contacts, message_template, attachment):
    messages = []
    for contact in contacts:
        msg = MIMEMultipart('alternative')

        contact = format_contact(contact)

        recipient_name = f"{contact['FIRST_NAME']} {contact['LAST_NAME']}"
        recipient_email = contact['EMAIL']
        
        msg['From'] = f"{your_name} <{your_email}>"
        msg['To'] = f"{recipient_name} <{recipient_email}>"
        msg['Subject'] = email_subject
        
        if message_template_file.endswith('.html') or message_template_file.endswith('.htm'):
            html_message = message_template.substitute(contact)
            text_maker = html2text.HTML2Text()
            text_maker.ignore_links = True
            text_maker.ignore_images = True
            text_maker.bypass_tables = True
            text_maker.ignore_emphasis = True
            plain_message = text_maker.handle(html_message)
            msg.attach(MIMEText(plain_message, 'plain'))
            msg.attach(MIMEText(html_message, 'html'))
        else:
            plain_message = message_template.substitute(contact)
            msg.attach(MIMEText(plain_message, 'plain'))
            
        if attachment_file:
            msg.attach(attachment_part(attachment_file))
        
        messages.append(msg)
    return messages

messages = create_messages(contacts, message_template, attachment_file)

## View messages (optional)

In [59]:
import time

for msg in messages:
    for part in msg.walk():
        content_type = part.get_content_maintype()
        
        # skip multipart containers and attachments
        if content_type != 'multipart' and content_type != 'application': 
            print(part)
        else:
            print(part['Content-Type'])
        time.sleep(0.2) # limit rate

multipart/alternative
Content-Type: text/plain; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

Dear Kamyar,

I am very excited to hear that you have been admitted to George Mason
University and are interested in pursuing a BS degree in Computer Engineering.
Our department has many outstanding faculty who are working on some amazing
research projects and, at the same time, are winning the highest awards that
the University and professional organizations have to offer for excellence in
teaching.

Whether you want to get involved in projects that involve swarming blimps,
robotic fish, machine learning, hardware security, 5G networks, satellite
communications, the power grid, or robotics, Electrical and Computer
Engineering is the place to be at Mason. I have attached a copy of the Fall
2019 ECE Newsletter that you may enjoy reading.

We all know and appreciate what a difficult and important decision it is to
select the right University for you to earn your Bachelor'

application/octet-stream


## Password

In [54]:
# we don't save the password
password = getpass.getpass('Password')


Password········


## Send

In [55]:
import smtplib
import ssl, time
from smtplib import SMTPException, SMTPServerDisconnected

send_delay = 3 # limit sending rate, Microsoft Office accounds have a maximum rate of 30 messages per minute

print(f"sending {len(messages)} emails...")
try:
    with smtplib.SMTP("smtp.office365.com", 587) as server:
        context = ssl.create_default_context()
        server.starttls(context=context)
        server.login(username, password)

        for msg in messages:
            try:
                server.send_message(msg)
            except SMTPServerDisconnected as e:
                print(f"SMTP disconnected. Reconnecting...")
                server.set_debuglevel(1)
                server.starttls(context=context)
                server.login(username, password)
                server.send_message(msg)
                server.set_debuglevel(0)

            print(f"Email sent to {msg.get('To')}")
            time.sleep(send_delay)

        print('All emails successfully sent!')
        server.quit()
except SMTPException as e:
    print(f"[ERROR] SMPT server connection error: {e}")
except ValueError as e:
    print(f"[ERROR] ValueError: {e}")
    raise
except Exception as e:
    print(f"[ERROR] exception received: {e}")
    raise
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

sending 3 emails...
Sent message to Kamyar Mohajerani <kammoh@gmail.com>
Sent message to Kamy M <kamyar@ieee.org>
Sent message to Kamyar Mohajerani <mmohajer@gmu.edu>
All emails successfully sent!
